In [13]:
from flask import Flask, request, Response
import os
import inference
import json

app = Flask(__name__)

def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']

    temp_dir = os.getenv("TEMP", "/tmp")  # Temporary directory for saving uploaded files
    file_path = os.path.join(temp_dir, file.filename)

    try:
        # Save the uploaded file temporarily
        file.save(file_path)

        # Call the inference method from the inference module
        example_model_path = "svm_model.pkl"  # Update as needed
        example_scaler_path = "scaler.pkl"   # Update as needed

        # Perform inference
        result = inference.inference(file_path, example_model_path, example_scaler_path)

        # Return result as JSON
        obj = {
            "result": np.int64(result),
        }
        return json.dumps(obj, default=np_encoder)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

    finally:
        # Clean up the file after processing
        if os.path.exists(file_path):
            os.remove(file_path)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://14.49.91.231:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2024 22:25:46] "POST /predict HTTP/1.1" 200 -
